## Reconstructions

In [3]:
# -------------
# I M P O R T S
# -------------

# MXNET:
import mxnet as mx
from mxnet import Context, cpu, gpu
from mxnet.ndarray import concat

# Other libraries:

from tqdm import tqdm
import matplotlib.pyplot as plt 
import numpy as np
import discriminator as dis
import time
from glob import glob

# MODULES:
from generator_FC import Generator
import modules as modules
from util import Device

In [4]:
runname = f'baseline_FC_1'
device = Device.GPU2
batch_size = 4
in_chan = 15
epoch = 6

In [5]:
context = cpu() if device.value == -1 else gpu(device.value)

with Context(context):

    test_iter = modules.make_iterator_preprocessed('testing', 'V1', 'V2', 'V3', batch_size=batch_size, shuffle=False)

    RF_signals_lengths = []
    for *RFsignals, targets in test_iter:
        for s in RFsignals:
            RF_signals_lengths.append(s.shape[2])
        break

    generator = Generator(in_chan, context, RF_in_units=RF_signals_lengths, 
                          conv_input_shape=(96, 96),
                              train_RF=True)
    
    generator.network.load_parameters(f'saved_models/{runname}/netG_{epoch}.model', ctx=context) 
    generator.rf_mapper.load_parameters(f'saved_models/{runname}/RFlayers_{epoch}.model')

    g = generator.network


In [6]:
# ====================
# T R AI N I N G
# ====================

outputs_lst = []
targets_lst = []
learned_inputs_lst = []
for *RFsignals, targets in tqdm(test_iter, total = len(test_iter)):
    with Context(context):
        # -----
        # Inputs
        # -----
        RFsignals = [x.as_in_context(context) for x in RFsignals]

        learned_inputs = generator.rf_mapper(RFsignals).reshape((-1,15, 96,96))

        # ------------------------------------
        # T R A I N  D i s c r i m i n a t o r
        # ------------------------------------
        targets = targets.as_in_context(context)

        outputs = g(learned_inputs).reshape((-1,3, 96,96))
        
    outputs_np = outputs.asnumpy()
    targets_np = targets.asnumpy()
    learned_inputs_np = learned_inputs.asnumpy()

    outputs_lst.append(outputs_np)
    targets_lst.append(targets_np)
    learned_inputs_lst.append(learned_inputs_np)

100%|██████████| 259/259 [00:03<00:00, 74.16it/s]


In [7]:
outputs = np.concatenate(outputs_lst)
targets = np.concatenate(targets_lst)
learned_inputs = np.concatenate(learned_inputs_lst)


In [8]:
np.savez_compressed(f'../final_reconstructions/FC_layers_model', learned_inputs = learned_inputs, targets = targets, outputs = outputs)